In [33]:
# Celda 1: Importaciones y Configuración Inicial (Versión Final Sincronizada)
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import yaml
import json
import os
import re # Añadimos 're' aquí para que esté disponible en todo el notebook

# Cargar la configuración del proyecto
with open('../config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# --- Rutas y Parámetros del Proyecto ---
MODEL_BASE_ID = config['student_model']['base_model_name']
experiment_v4_config = config['experiments']['v4_silver_balanced_reasoning']

# Ruta al modelo v4 (corregida con 'final_checkpoint')
lora_base_path = os.path.join('..', experiment_v4_config['output_dir'])
LORA_ADAPTER_PATH_V4 = os.path.join(lora_base_path, 'final_checkpoint')

# Ruta a los datos de test (JSONL con los prompts completos)
TEST_DATA_PATH_JSONL = os.path.join('..', config['data_paths']['gold_standard']['test_jsonl'])

# --- ¡ACTUALIZADO! Ruta al CSV con los resultados de la evaluación ---
EVAL_RESULTS_PATH_CSV = '../outputs/reports/evaluation_details_v4_silver_balanced_reasoning.csv' 

print("✅ Configuración cargada.")
print(f"   - Modelo Base: {MODEL_BASE_ID}")
print(f"   - Adaptadores LoRA (v4): {LORA_ADAPTER_PATH_V4}")
print(f"   - Datos de Test (JSONL): {TEST_DATA_PATH_JSONL}")
print(f"   - Resultados de Evaluación (CSV): {EVAL_RESULTS_PATH_CSV}")

✅ Configuración cargada.
   - Modelo Base: Qwen/Qwen3-4B-Instruct-2507
   - Adaptadores LoRA (v4): ..\models/distilmatch_v4_silver_balanced_reasoning\final_checkpoint
   - Datos de Test (JSONL): ..\data/03_gold_standard/gold_standard_test.jsonl
   - Resultados de Evaluación (CSV): ../outputs/reports/evaluation_details_v4_silver_balanced_reasoning.csv


In [6]:
# Celda 2: Cargar el Modelo y el Tokenizer

print("⏳ Cargando modelo y tokenizer... (Puede tardar unos minutos)")

# Configuración de cuantización en 4-bit (consistente con el entrenamiento)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Cargar el modelo base desde Hugging Face Hub
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_BASE_ID,
    quantization_config=bnb_config,
    device_map="auto" # Automáticamente usa la GPU si está disponible
)

# Cargar el tokenizer asociado al modelo base
tokenizer = AutoTokenizer.from_pretrained(MODEL_BASE_ID)
# Es importante asegurarse de que el pad_token es el eos_token para modelos decoder-only
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Cargar y fusionar los adaptadores LoRA del modelo v4
# PeftModel es la librería que se encarga de aplicar los adaptadores al modelo base.
model_v4 = PeftModel.from_pretrained(base_model, LORA_ADAPTER_PATH_V4)

print("✅ ¡Modelo v4 listo para la inferencia!")

⏳ Cargando modelo y tokenizer... (Puede tardar unos minutos)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ ¡Modelo v4 listo para la inferencia!


In [7]:
# Celda 3: Cargar los Datos de Test

# Cargar el archivo JSONL en un DataFrame de pandas para facilitar la manipulación
test_df = pd.read_json(TEST_DATA_PATH, lines=True)

print(f"✅ Cargados {len(test_df)} ejemplos del conjunto de test.")
print("Vista previa de los datos:")
test_df.head()

✅ Cargados 44 ejemplos del conjunto de test.
Vista previa de los datos:


,prompt,response
0,You are an automated scoring system. Your SOLE...,Score: 45.0
1,You are an automated scoring system. Your SOLE...,Score: 70.0
2,You are an automated scoring system. Your SOLE...,Score: 70.0
3,You are an automated scoring system. Your SOLE...,Score: 15.0
4,You are an automated scoring system. Your SOLE...,Score: 45.0


In [35]:
# Celda 4: La Función de Inferencia "Thinking Mode" (Fiel a la Evaluación)

def get_v4_prediction(cv_text: str, job_description_text: str, model, tokenizer):
    """
    Toma un CV y una oferta, y genera el razonamiento JSON usando el formato exacto
    del pipeline de evaluación.
    """
    # 1. System Prompt: Exactamente como en tu script de evaluación.
    system_prompt = """As an expert HR analyst, provide a step-by-step analysis of the compatibility between the following CV and Job Offer. 
Your response must be a JSON object with the following structure:
{
    "strengths": "A brief analysis of the candidate's strengths.",
    "concerns_and_gaps": "A brief analysis of the candidate's weaknesses or gaps.",
    "verdict": "One of the following categories: 'MUST INTERVIEW', 'PROMISING FIT', 'BORDERLINE', 'NO FIT'.",
    "score": A numerical score from 0 to 100.
}
The JSON object should be enclosed in ```json ... ```."""

    # 2. User Prompt: Usa los placeholders {cv} y {job_description}.
    user_prompt_template = """[CV]
{cv}

[OFERTA DE TRABAJO]
{job_description}"""

    # 3. Rellenamos la plantilla del usuario.
    final_user_prompt = user_prompt_template.format(cv=cv_text, job_description=job_description_text)
    
    # 4. Construimos el formato de chat que espera el modelo.
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": final_user_prompt}
    ]
    
    # 5. Preparamos y ejecutamos la inferencia.
    full_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(full_prompt, return_tensors="pt", return_attention_mask=True).to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=400, pad_token_id=tokenizer.eos_token_id)
    raw_output = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    
    # 6. Extraemos y parseamos el JSON de la respuesta.
    try:
        # Buscamos el bloque de código JSON ```json ... ```
        json_match = re.search(r"```json\s*\n(.*?)\n\s*```", raw_output, re.DOTALL)
        if json_match:
            json_str = json_match.group(1)
            parsed_json = json.loads(json_str)
            return parsed_json, raw_output
        else:
            # Si no encuentra el bloque, intenta parsear la salida cruda
            parsed_json = json.loads(raw_output)
            return parsed_json, raw_output
            
    except (json.JSONDecodeError, AttributeError):
        return {"error": "Failed to parse JSON from model output"}, raw_output

In [ ]:
# Celda 5: El Laboratorio de Análisis (Versión Final y Sincronizada)

# --- 1. CARGAMOS LOS DOS ARCHIVOS DE DATOS ---
# El CSV contiene los scores que ya calculaste.
eval_df = pd.read_csv(EVAL_RESULTS_PATH_CSV)

# El JSONL contiene los prompts originales con los textos completos.
# Es importante que las filas de ambos archivos estén en el mismo orden.
prompts_df = pd.read_json(TEST_DATA_PATH_JSONL, lines=True)


# --- 2. SELECCIONA EL CASO A ESTUDIAR ---
index_to_analyze = 0 # <--- ¡CAMBIA ESTE NÚMERO PARA EXPLORAR!


# 3. EXTRAE LA INFORMACIÓN DE CADA FUENTE USANDO EL MISMO ÍNDICE
try:
    # De nuestro CSV, cogemos los scores.
    sample_from_csv = eval_df.iloc[index_to_analyze]
    true_score = sample_from_csv['true_score']
    predicted_score_from_csv = sample_from_csv['predicted_score']

    # De nuestro JSONL, cogemos el prompt para extraer los textos.
    sample_from_jsonl = prompts_df.iloc[index_to_analyze]
    prompt_full_text = sample_from_jsonl['prompt']

    # Extraemos los textos del CV y la Oferta usando el método split() que ya validamos.
    text_after_cv_tag = prompt_full_text.split('[CV]')[1]
    parts = text_after_cv_tag.split('[OFERTA DE TRABAJO]')
    cv_text = parts[0].strip()
    # ¡Importante! Usamos 'job_description_text' para que coincida con la función de la Celda 4.
    job_description_text = parts[1].strip()

    # --- 4. MOSTRAMOS EL ANÁLISIS ---
    print("="*60)
    print(f"🔬 ANALIZANDO CASO DE ESTUDIO (Índice: {index_to_analyze})")
    print("="*60)
    print(f"🎯 Score Real (Humano):          {true_score}")
    print(f"📊 Score Predicho (del CSV):     {predicted_score_from_csv}")
    print("-" * 60)
    print("⏳ Generando razonamiento en vivo del modelo v4 para este caso...")

    # 5. Generamos el razonamiento en vivo llamando a la función de la Celda 4
    predicted_json_live, raw_output_live = get_v4_prediction(
        cv_text=cv_text, 
        job_description_text=job_description_text, 
        model=model_v4, 
        tokenizer=tokenizer
    )

    # 6. Mostramos el razonamiento detallado
    print("\n" + "="*60)
    print("🤖 RAZONAMIENTO EN VIVO DEL MODELO v4")
    print("="*60)

    if "error" in predicted_json_live:
        print("❌ ¡ERROR! El modelo no generó un JSON válido en esta ejecución.")
        print("\n--- Salida Cruda del Modelo ---")
        print(raw_output_live)
    else:
        print(f"📊 Score (en vivo):              {predicted_json_live.get('score', 'N/A')}")
        print(f"⚖️ Veredicto (en vivo):          {predicted_json_live.get('verdict', 'N/A')}")
        print("\n✅ Fortalezas Identificadas:")
        print(f"   {predicted_json_live.get('strengths', 'N/A')}")
        print("\n🤔 Preocupaciones y Brechas Identificadas:")
        print(f"   {predicted_json_live.get('concerns_and_gaps', 'N/A')}")

except IndexError:
    print(f"❌ Error: El índice {index_to_analyze} está fuera de rango. Elige un número entre 0 y {len(eval_df)-1}.")
except Exception as e:
    print(f"❌ Ocurrió un error inesperado durante el análisis: {e}")


🔬 ANALIZANDO CASO DE ESTUDIO (Índice: 0)
🎯 Score Real (Humano):          45.0
📊 Score Predicho (del CSV):     85.0
------------------------------------------------------------
⏳ Generando razonamiento en vivo del modelo v4 para este caso...

🤖 RAZONAMIENTO EN VIVO DEL MODELO v4
📊 Score (en vivo):              55
⚖️ Veredicto (en vivo):          BORDERLINE

✅ Fortalezas Identificadas:
   The candidate has over 4 years of experience in data analysis and business intelligence, which aligns well with the core responsibilities of a Data Analyst. Their proficiency in Python, Excel, and data visualization tools suggests a strong analytical foundation. Additionally, their background in machine learning and AI indicates a technical aptitude that could be beneficial in a data-driven role.

🤔 Preocupaciones y Brechas Identificadas:
   The candidate's experience is significantly mismatched with the job title and level. The role is explicitly an Entry-Level Business Analyst/Product Owner requiring 

: 